In [95]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import numpy as np
import pandas as pd
import re

In [ ]:
url= 'https://www.asn.fr/l-asn-controle/actualites-du-controle/installations-nucleaires/arret-de-reacteurs-de-centrales-nucleaires?page=1&publication_date_year%5Bfrom%5D=2011&publication_date_year%5Bto%5D=2022'
request = requests.request('GET',url)
soup = BeautifulSoup(request.content)
nb_pages = int(soup.find_all('a',{'class': 'page-link'})[-2].text)
list_rn = []
art_tot = 1
for i in range(1,nb_pages+1):
    #print(i)
    if i != 1:
        url= 'https://www.asn.fr/l-asn-controle/actualites-du-controle/installations-nucleaires/arret-de-reacteurs-de-centrales-nucleaires?page={}&publication_date_year%5Bfrom%5D=2011&publication_date_year%5Bto%5D=2022'.format(i)
        request = requests.request('GET',url)
        soup = BeautifulSoup(request.content)
    main_content = soup.find('div', id='main-content')
    for art in range(len(main_content.find_all('span', {'class':'Tooltip-title'}))):
        centrale = main_content.find_all('span', {'class':'Tooltip-title'})[art].text
        reacteur = re.findall(r"[0-9+]",main_content.find_all('div', {'class':'eztext-field'})[art].text)[0]
        #print(centrale, reacteur)
        url= 'https://www.asn.fr' + main_content.find_all('a', attrs={'class':'Teaser-titleLink'})[art]['href']
        request = requests.request('GET',url)
        soup = BeautifulSoup(request.content)
        try:
            date_arret = soup.find_all('b')[0].text
            date_redem = soup.find_all('b')[1].text
            list_rn.append([centrale, reacteur, date_arret, date_redem])
        except:
            print('Erreur')

        art_tot += 1

#print(art_tot)
ar_rn = np.array(list_rn)

Trouver le nombre de pages sur la page principale (1 à 36 au 10 Janvier 2023)
Boucle : sur toutes les pages (reconnexion à des url)
    Boucle: Pour chaque publication d'avis de maintenance
        1ère étape : Scrappe la partie générale pour obtenir la centrales nucléaires, numéro de réacteur, url spécifique
        2ème étape : aller sur l'url spécifique (reconnexion àn des url)

In [162]:
df_rn = pd.DataFrame(ar_rn, columns=['centrale_nucleaire','no_reacteur', 'date_arret', 'date_redem' ])

In [163]:
df_rn['centrale_nucleaire'] = df_rn['centrale_nucleaire'].apply(lambda x: x.lower())

In [164]:
#df_rn['centrale_nucleaire'].apply(lambda x: re.sub(r'({centrale nucléaire de})($)' , r'\2', x))

In [165]:
df_rn['centrale_reacteur'] = df_rn['centrale_nucleaire']+'_reacteur_'+df_rn['no_reacteur']

In [166]:
df_rn = df_rn.drop(['centrale_nucleaire', 'no_reacteur'], axis=1)

In [ ]:
df_rn['date_arret']=df_rn['date_arret'].apply(lambda x: pd.to_datetime(x))
df_rn['date_redem']=df_rn['date_redem'].apply(lambda x: pd.to_datetime(x))

In [171]:
sort_ar=df_rn['date_arret'].sort_values().reset_index(drop=True)
sort_red = df_rn['date_redem'].sort_values(ascending=False).reset_index(drop=True)

In [235]:
#today = pd.to_datetime("today")
date_index = pd.date_range(start=sort_ar[0], end=sort_red[0], freq='D')
date_index_hourly = pd.date_range(start=sort_ar[0], end=sort_red[0], freq='h')

In [210]:
rn_off=[]
for day in date_index:
    check = (df_rn['date_arret']<day)&(day<df_rn['date_redem'])
    ind = check.value_counts().index
    if True in ind:
        off = check.value_counts()[1]
    else:
        off=0
    rn_off.append([day, off])
rn_off_ar = np.array(rn_off)
df_arr = pd.DataFrame(rn_off, columns=['date', 'reacteurs_off'])

In [231]:
df_arr.loc[df_arr['date']==pd.to_datetime('11/11/2022')]

,date,reacteurs_off
7494,2022-11-11,12


In [243]:
df_arr

,date,reacteurs_off
0,2002-05-06,0
1,2002-05-07,1
2,2002-05-08,1
3,2002-05-09,1
4,2002-05-10,1
...,...,...
13038,2038-01-15,2
13039,2038-01-16,2
13040,2038-01-17,2
13041,2038-01-18,1


In [239]:
rn_off_hourly=[]
for hour in date_index_hourly:
    check = (df_rn['date_arret']<hour)&(hour<df_rn['date_redem'])
    ind = check.value_counts().index
    if True in ind:
        off = check.value_counts()[1]
    else:
        off=0
    rn_off_hourly.append([hour, off])
rn_off_ar_h = np.array(rn_off_hourly)
df_arr_hourly = pd.DataFrame(rn_off_hourly, columns=['date', 'reacteurs_off'])

In [240]:
df_arr_hourly

,date,reacteurs_off
0,2002-05-06 00:00:00,0
1,2002-05-06 01:00:00,1
2,2002-05-06 02:00:00,1
3,2002-05-06 03:00:00,1
4,2002-05-06 04:00:00,1
...,...,...
313004,2038-01-18 20:00:00,1
313005,2038-01-18 21:00:00,1
313006,2038-01-18 22:00:00,1
313007,2038-01-18 23:00:00,1
